In [2]:
from coffea.nanoevents import BaseSchema, NanoEventsFactory
import dask_awkward as dak
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import hist
import mplhep as hep
import numba
import vector
vector.register_awkward()

In [3]:
events = NanoEventsFactory.from_root(
    file="../data/p8_ee_ZH_ecm240/events_101027117.root:events",
    schemaclass=BaseSchema,
    # schemaclass=DelphesSchema, Doesn't work 
    # schemaclass=NanoAODSchema.v7, Doesn't work
    metadata={
        "path":"/eos/experiment/fcc/ee/generation/DelphesEvents/spring2021/IDEA/p8_ee_ZH_ecm240/events_101027117.root",
        "name":"p8_ee_ZH_ecm240",
        "process":"ZH to mumu, ecm=240 GeV"
    }
).events()

In [4]:
from coffea.analysis_tools import PackedSelection

In [5]:
def index_mask(input_array, index_array):
    '''
    This function matches the given attribute of ReconstructedParticles (for example energy) to the particle index (for example Muon or Electron)
    '''
    # input_array = input_array.compute()
    # index_array = index_array.compute()
    if len(input_array) != len(index_array) :
        raise Exception(f'Length of input_array({len(input_array)}) and index_array({len(index_array)}) does not match!')
    counts = len(index_array)
    
    @numba.jit
    def numba_wrap(input_array, index_array, counts):
        output_array = []
        for event_index in range(counts):
            event_mask = index_array[event_index]
            reco_list = input_array[event_index]
            output_array.append([reco_list[i] for i in  event_mask])
        return output_array
    out = ak.Array(numba_wrap(input_array,index_array,counts))
    
    return out

In [6]:
cut = PackedSelection()

# Filter out any event with no reconstructed particles
Recon = events['ReconstructedParticles/ReconstructedParticles.energy'].compute()
useful_events = events[ak.num(Recon) > 0]


# Generate Reconstructed Particle Attributes
Reco_E = useful_events['ReconstructedParticles/ReconstructedParticles.energy'].compute()
Reco_px = useful_events['ReconstructedParticles/ReconstructedParticles.momentum.x'].compute()
Reco_py = useful_events['ReconstructedParticles/ReconstructedParticles.momentum.y'].compute()
Reco_pz = useful_events['ReconstructedParticles/ReconstructedParticles.momentum.z'].compute()
Reco_q = useful_events['ReconstructedParticles/ReconstructedParticles.charge'].compute()
Reco_mass = useful_events['ReconstructedParticles/ReconstructedParticles.mass'].compute()
cut.add('At least one Reconstructed Particle', ak.all(Reco_E > 0, axis=1))#.compute())

# Generate Muon Attributes
Muon_index = useful_events['Muon#0/Muon#0.index'].compute()
Muon_E = index_mask(Reco_E,Muon_index)
Muon_px = index_mask(Reco_px,Muon_index)
Muon_py = index_mask(Reco_py,Muon_index)
Muon_pz = index_mask(Reco_pz,Muon_index)
Muon_q = index_mask(Reco_q,Muon_index)
Muon_mass = index_mask(Reco_mass,Muon_index)

# Create Array of Muon Lorentz Vector 
Muon = ak.zip({"px":Muon_px,"py":Muon_py,"pz":Muon_pz,"E":Muon_E,"q":Muon_q,}, with_name="Momentum4D")

# Muon pt > 10
Muon_pt_cut = ak.all(Muon.pt > 10, axis=1)
Muon = ak.mask(Muon, Muon_pt_cut) #ak.mask to preserve number of events
cut.add('Muon $p_T$ > 10 [GeV]',Muon_pt_cut)

# Produce all the combinations of Muon Pairs possible within an event
combs = ak.combinations(Muon,2)

# Get dimuons
mu1 , mu2 = ak.unzip(combs)
di_muon = mu1 + mu2

# Selection 0 : Only one Z candidate in an event
di_muon = ak.mask(di_muon, ak.num(di_muon) == 1)
di_muon_mass = ak.flatten(di_muon.mass)
cut.add('$N_Z$',ak.num(Muon) == 2 ) #Having one Z candidate is same as having exactly two muons in an event
# di_muon_mass = ak.Array([i[0] for i in ak.sort(di_muon.mass, ascending=False)])

# Choose dimuon which is made up of two oppositely charged muons
q_sum = mu1.q + mu2.q
q_sum_array = ak.mask(q_sum, ak.num(q_sum) == 1)
q_sum_mask = ak.all(q_sum_array == 0, axis=1)
Z_cand = ak.mask(di_muon , q_sum_mask)
cut.add('Opp charge muons',q_sum_mask)

#Recoil Calculation
ecm = 240 #GeV # com energy
initial = ak.zip({"px":0,"py":0,"pz":0,"E":ecm}, with_name="Momentum4D")
Recoil = initial - Z_cand

# Selection 1 : Selection 0 + 80 < M_Z < 100
zmassmask = (Z_cand.mass > 80) & (Z_cand.mass < 100)
Z_cand_sel1 = ak.mask(Z_cand, zmassmask)
Recoil_sel1 = ak.mask(Recoil, zmassmask)
print(zmassmask)
zmassmask = ak.fill_none(zmassmask,[False],axis=0)
zmassmask = ak.flatten(zmassmask)
cut.add('80 < $M_Z$ < 100',zmassmask)

[None, None, None, None, None, None, ..., None, None, None, None, None, None]


In [7]:
cut.names

['At least one Reconstructed Particle',
 'Muon $p_T$ > 10 [GeV]',
 '$N_Z$',
 'Opp charge muons',
 '80 < $M_Z$ < 100']

In [8]:
c = cut.cutflow(*cut.names)

In [9]:
onecut, cutflow, labels, = c.yieldhist()

In [10]:
onecut

Hist(Integer(0, 6, name='onecut'), storage=Double()) # Sum: 304958.0

In [11]:
cutflow

Hist(Integer(0, 6, name='cutflow'), storage=Double()) # Sum: 304958.0

In [12]:
labels

['initial',
 'At least one Reconstructed Particle',
 'Muon $p_T$ > 10 [GeV]',
 '$N_Z$',
 'Opp charge muons',
 '80 < $M_Z$ < 100']

In [13]:
a = ak.Array([[1,2,None],None,[None]])

In [14]:
a

<Array [[1, 2, None], None, [None]] type='3 * option[var * ?int64]'>

In [15]:
a_cut = a > 1
a_cut

<Array [[False, True, None], None, [None]] type='3 * option[var * ?bool]'>

In [16]:
a_cut = ak.fill_none(a_cut, False)
a_cut

<Array [[False, True, False], None, [False]] type='3 * option[var * bool]'>

In [17]:
b = ak.fill_none(a_cut,[False], axis=0)
b

<Array [[False, True, False], [False], [False]] type='3 * var * bool'>

In [18]:
ak.flatten(b)

<Array [False, True, False, False, False] type='5 * bool'>

In [19]:
from coffea.util import load

In [20]:
input = load('./outputs/FCCee/higgs/mH-recoil/mumu/mHrecoil_mumu.coffea')

In [21]:
input['p8_ee_ZH_ecm240'].keys()

dict_keys(['histograms', 'cutflow'])

In [22]:
c_zh = input['p8_ee_ZH_ecm240']['cutflow']['sel0']
c_zz = input['p8_ee_ZZ_ecm240']['cutflow']['sel0']
c_ww = input['p8_ee_WW_ecm240']['cutflow']['sel0']
c_zh

Cutflow(selections=('At least one Reconstructed Particle', 'Muon $p_T$ > 10 [GeV]', '$N_Z$', 'Opp charge muons'))

In [23]:
res_zh = c_zh.result()
res_zz = c_zz.result()
res_ww = c_ww.result()
res_zh

CutflowResult(labels=['initial', 'At least one Reconstructed Particle', 'Muon $p_T$ > 10 [GeV]', '$N_Z$', 'Opp charge muons'], nevonecut=[499919, 499919, 485621, 13808, 13692], nevcutflow=[499919, 499919, 485621, 13808, 13692], masksonecut=[array([ True,  True,  True, ...,  True,  True,  True]), array([ True,  True,  True, ...,  True,  True,  True]), array([False, False, False, ..., False, False, False]), array([False, False, False, ..., False, False, False])], maskscutflow=[array([ True,  True,  True, ...,  True,  True,  True]), array([ True,  True,  True, ...,  True,  True,  True]), array([False, False, False, ..., False, False, False]), array([False, False, False, ..., False, False, False])])

In [24]:
import numba

In [25]:
def add_masks(mask_list):
    new_list = [np.array(mask) for mask in mask_list]
    # bigm = ak.Array(new_list)
    return sum(new_list)

In [26]:
res_zz.maskscutflow

[array([ True,  True,  True, ...,  True,  True,  True]),
 array([ True,  True,  True, ...,  True,  True,  True]),
 array([False, False,  True, ..., False, False, False]),
 array([False, False,  True, ..., False, False, False])]

In [27]:
res_ww.maskscutflow

[array([ True,  True,  True, ...,  True,  True,  True]),
 array([ True,  True,  True, ...,  True,  True,  True]),
 array([False, False, False, ..., False, False, False]),
 array([False, False, False, ..., False, False, False])]

In [28]:
add_masks([res_zz.maskscutflow, res_ww.maskscutflow])

ValueError: operands could not be broadcast together with shapes (4,488035) (4,499994) 

In [29]:
mask_list = [res_zz.maskscutflow, res_ww.maskscutflow]
mask_list

[[array([ True,  True,  True, ...,  True,  True,  True]),
  array([ True,  True,  True, ...,  True,  True,  True]),
  array([False, False,  True, ..., False, False, False]),
  array([False, False,  True, ..., False, False, False])],
 [array([ True,  True,  True, ...,  True,  True,  True]),
  array([ True,  True,  True, ...,  True,  True,  True]),
  array([False, False, False, ..., False, False, False]),
  array([False, False, False, ..., False, False, False])]]

In [30]:
sum_axis_0 = np.zeros_like(mask_list[0])
print(sum_axis_0)
for mask in mask_list:
    np_mask = np.array(mask)
    print(np_mask.shape)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
(4, 488035)
(4, 499994)


In [31]:
import hist

In [32]:
h = hist.Hist.new.Reg(100,0,200).Double().fill(ak.flatten(Muon_E))
h

Hist(Regular(100, 0, 200, label='Axis 0'), storage=Double()) # Sum: 14892.0

In [33]:
sum([h,h,h,h])

Hist(Regular(100, 0, 200, label='Axis 0'), storage=Double()) # Sum: 59568.0

In [34]:
import collections

In [35]:
a = collections.namedtuple('cutflow', ['histo'])

In [36]:
b = a(h)

In [37]:
b

cutflow(histo=Hist(Regular(100, 0, 200, label='Axis 0'), storage=Double()) # Sum: 14892.0)

In [38]:
b.histo

Hist(Regular(100, 0, 200, label='Axis 0'), storage=Double()) # Sum: 14892.0

In [39]:
c_zh.yieldhist()

(Hist(Integer(0, 5, name='onecut'), storage=Double()) # Sum: 1512959.0,
 Hist(Integer(0, 5, name='cutflow'), storage=Double()) # Sum: 1512959.0,
 ['initial',
  'At least one Reconstructed Particle',
  'Muon $p_T$ > 10 [GeV]',
  '$N_Z$',
  'Opp charge muons'])

In [40]:
(a,b,c) = c_zh.yieldhist()

In [41]:
a

Hist(Integer(0, 5, name='onecut'), storage=Double()) # Sum: 1512959.0

In [42]:
b

Hist(Integer(0, 5, name='cutflow'), storage=Double()) # Sum: 1512959.0

In [43]:
c

['initial',
 'At least one Reconstructed Particle',
 'Muon $p_T$ > 10 [GeV]',
 '$N_Z$',
 'Opp charge muons']

In [44]:
c_zh.result().labels

['initial',
 'At least one Reconstructed Particle',
 'Muon $p_T$ > 10 [GeV]',
 '$N_Z$',
 'Opp charge muons']

In [45]:
b

Hist(Integer(0, 5, name='cutflow'), storage=Double()) # Sum: 1512959.0

In [54]:
l, = b.axes.edges.tolist()

In [55]:
l

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]

In [60]:
b.axes.edges[0][-1]

5.0

In [63]:
np.array(l).tolist()

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]

In [64]:
events

dask.awkward<from-uproot, npartitions=1>

In [77]:
Recon = events['ReconstructedParticles/ReconstructedParticles.energy']
dak.num(dak.sum(Recon, axis = 1), axis=0).compute()

100000

In [78]:
dak.num(Recon, axis=0).compute()

100000

In [74]:
events['ReconstructedParticles/ReconstructedParticles.energy'].compute()

ArrayType(ListType(NumpyType('float32', parameters={'__doc__': 'energy[ReconstructedParticles_]'}), parameters={'__doc__': 'energy[ReconstructedParticles_]'}), 100000, {'Systematic': <class 'coffea.nanoevents.methods.base.SystematicRecord'>, ('*', 'Systematic'): <class 'coffea.nanoevents.methods.base.SystematicArray'>, ('__typestr__', 'Systematic'): 'Systematic', 'UpDownSystematic': <class 'coffea.nanoevents.methods.systematics.UpDownSystematic.UpDownSystematicRecord'>, ('*', 'UpDownSystematic'): <class 'coffea.nanoevents.methods.systematics.UpDownSystematic.UpDownSystematicArray'>, ('__typestr__', 'UpDownSystematic'): 'UpDownSystematic', 'NanoEvents': <class 'coffea.nanoevents.methods.base.NanoEventsRecord'>, ('*', 'NanoEvents'): <class 'coffea.nanoevents.methods.base.NanoEventsArray'>, ('__typestr__', 'NanoEvents'): 'event', 'NanoCollection': <class 'coffea.nanoevents.methods.base.NanoCollectionRecord'>, ('*', 'NanoCollection'): <class 'coffea.nanoevents.methods.base.NanoCollectionAr